In [5]:
import torch as t
#print(torch.__version__)
#print(torch.cuda.is_available())  # should be False on macOS
#print(torch.backends.mps.is_available())  # True if Apple Silicon

In [2]:
def to_onehot(y, num_classes):
    y_onehot = torch.zeros(y.size(0), num_classes)
    y_onehot.scatter_(1, y.view(-1, 1).long(), 1).float()
    return y_onehot

y = torch.tensor([0, 1, 2, 2])

y_enc = to_onehot(y, 3)

print('one-hot encoding:\n', y_enc)

one-hot encoding:
 tensor([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.]])


In [4]:
t0 = t.tensor(1)

In [11]:
t1 = t.tensor([1,2,3])
t2 = t.tensor([[1,2],[3,4]])
t3 = t.tensor([[[1,2],[3,4]],
               [[5,6], [7,8]]])

In [12]:
print(t1.dtype)

torch.int64


In [13]:
floatvec = t1.to(t.float32)

In [14]:
print(floatvec)

tensor([1., 2., 3.])


In [18]:
t22 = t.tensor([[1,2,3], [4,5,6]])

In [19]:
print(t22.reshape(3,2))

tensor([[1, 2],
        [3, 4],
        [5, 6]])


In [20]:
print(t22.T)

tensor([[1, 4],
        [2, 5],
        [3, 6]])


In [21]:
print(t2.matmul(t2.T))

tensor([[ 5, 11],
        [11, 25]])


In [6]:
import torch.nn.functional as F
y = t.tensor([1.0])
x1 = t.tensor([1.1])
w1 = t.tensor([2.2])
b = t.tensor([0.0])
z = x1 * w1 + b
a = t.sigmoid(z)
loss = F.binary_cross_entropy(a,y)

In [7]:
loss

tensor(0.0852)

In [9]:
import torch.nn.functional as F
from torch.autograd import grad
y = t.tensor([1.0])
x1 = t.tensor([1.1])
w1 = t.tensor([2.2], requires_grad=True)
b = t.tensor([0.0], requires_grad= True)
z = x1 * w1 + b
a = t.sigmoid(z)
loss = F.binary_cross_entropy(a,y)
grad_l_w1 = grad(loss, w1, retain_graph = True)
grad_L_b = grad(loss, b, retain_graph = True)

In [10]:
print(grad_L_b, grad_l_w1)

(tensor([-0.0817]),) (tensor([-0.0898]),)


In [ ]:
print(grad_L_b, grad_l_w1)

(tensor([-0.0817]),) (tensor([-0.0898]),)


In [11]:
class NeuralNetwork(t.nn.Module):
    def __init__(self, num_ins, num_outs):
        super().__init__()
        self.layers = t.nn.Sequential(
            # 1st layer 
            t.nn.Linear(num_ins,30),
            t.nn.ReLU(),

            #2nd layer
            t.nn.Linear(30,20),
            t.nn.ReLU(),
            #output layer
            t.nn.Linear(20, num_outs),
        )
    
    def forward(self, x):
        logs = self.layers(x)
        return logs


In [12]:
model = NeuralNetwork(50,3)

In [13]:
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)

2213


In [16]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [17]:
t.manual_seed(123)
model = NeuralNetwork(50,3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [18]:
t.manual_seed(123)
X = t.rand((1,50))
out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]], grad_fn=<AddmmBackward0>)


In [19]:
with t.no_grad():
    out = t.softmax(model(X), dim=1)
print(out)

tensor([[0.3113, 0.3934, 0.2952]])


In [20]:
X_train = t.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = t.tensor([0,0,0,1,1])
X_test = t.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = t.tensor([0,1])

In [22]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features =X
        self.labels = y
    def __getitem__(self, index):
        o_x = self.features[index]
        o_y = self.labels[index]
        return o_x, o_y
    def __len__(self):
        return self.labels.shape[0]
    
train_ds = ToyDataset(X_train,y_train)
test_ds = ToyDataset(X_test, y_test)

In [23]:
print(len(train_ds))

5


In [24]:
from torch.utils.data import DataLoader
t.manual_seed(123)
train_loader = DataLoader(
    dataset= train_ds,
    batch_size=2,
    shuffle= True,
    num_workers=0
)
test_loader = DataLoader(
    dataset= test_ds,
    batch_size=2,
    shuffle= False,
    num_workers=0
)

In [25]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [26]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])
Batch 2: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 3: tensor([[ 2.7000, -1.5000]]) tensor([1])


In [30]:
import torch.nn.functional as F
t.manual_seed(123)
model = NeuralNetwork(num_ins=2,num_outs=2)
optimizer = t.optim.SGD(
    model.parameters(), lr=0.5
)
num_epochs =3
for epoch in range(num_epochs):
    model.train()
    for batch_idx,( features, labels) in enumerate(train_loader):
        log = model(features)
        loss = F.cross_entropy(log, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    ### Logging
        print(f"Epoch:{epoch+1:03d}/{num_epochs:03d}"
          f"|Batch{batch_idx:03d}/{len(train_loader):03d}"
          f"|Train Loss:{loss:.2f}")
model.eval()

Epoch:001/003|Batch000/003|Train Loss:0.75
Epoch:001/003|Batch001/003|Train Loss:0.65
Epoch:001/003|Batch002/003|Train Loss:0.42
Epoch:002/003|Batch000/003|Train Loss:0.05
Epoch:002/003|Batch001/003|Train Loss:0.13
Epoch:002/003|Batch002/003|Train Loss:0.00
Epoch:003/003|Batch000/003|Train Loss:0.01
Epoch:003/003|Batch001/003|Train Loss:0.00
Epoch:003/003|Batch002/003|Train Loss:0.02


NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=2, bias=True)
  )
)

In [32]:
with t.no_grad():
    outs  = model(X_train)
print(outs)

tensor([[ 2.9320, -4.2563],
        [ 2.6045, -3.8389],
        [ 2.1484, -3.2514],
        [-2.1461,  2.1496],
        [-2.5004,  2.5210]])


In [33]:
t.set_printoptions(sci_mode=False)
probs = t.softmax(outs, dim=1)
print(probs)

tensor([[    0.9992,     0.0008],
        [    0.9984,     0.0016],
        [    0.9955,     0.0045],
        [    0.0134,     0.9866],
        [    0.0066,     0.9934]])


In [34]:
preds = t.argmax(probs, dim=1)
print(preds)

tensor([0, 0, 0, 1, 1])


In [35]:
preds == y_train

tensor([True, True, True, True, True])

In [40]:
def compute_accuracy(model, dataloader):
    model = model.eval()
    correct = 0.0
    t_example = 0

    for idx, (features, labels) in enumerate(dataloader):
        with t.no_grad():
            logs = model(features)
        pred =t.argmax(logs,dim=1)
        compare = labels == pred
        correct += t.sum(compare)
        t_example += len(compare)
    return (correct/t_example).item()

In [41]:
print(compute_accuracy(model,train_loader))

1.0


In [42]:
t.save(model.state_dict(), "model.pth")

In [ ]:
m1 = NeuralNetwork(2,2)
m1.load_state_dict(t.load("model.pth"))

In [43]:
t1 = t.tensor([1,2,3])
t2 = t.tensor([4,5,6])
print(t1+t2)

tensor([5, 7, 9])


In [44]:
t1 = t1.to("cuda")
t2 = t2.to("cuda")
print(t1+t2)

AssertionError: Torch not compiled with CUDA enabled

In [45]:
t.cuda.is_available()

False

In [46]:
device = t.device("mps" if t.backends.mps.is_available() else "cpu")
device

device(type='mps')